In [ ]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('../cmake-build-release')

import pymgodpl as mg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time

In [ ]:
rm = mg.load_robot_model(1.0)

In [ ]:
trees = mg.load_all_tree_meshes()

In [ ]:
sample_data = []
N_TRIALS = 1000
MAX_FRUIT = 500

for tree in trees:
    
    fruit = tree.fruit_positions()
    
    print(f'Starting tree: {tree.name} with {len(fruit)} fruit')
    
    if len(fruit) > MAX_FRUIT:
        fruit = random.sample(fruit, MAX_FRUIT)
    
    cd = mg.CollisionDetection([tree.trunk], rm)
    
    for (fruit_i, fruit) in enumerate(fruit):
        
        successes = 0
        
        for sample_i in range(N_TRIALS):

            sample = mg.sample_goal_region(fruit, random.randint(0, 100000), rm)
            collides = cd.collides(sample)
            
            if not collides:
                successes += 1
                
        sample_data.append({'tree':tree.name, 'successes':successes})
    
    print('Done with tree: ', tree.name)
            
sample_data = pd.DataFrame(sample_data)

In [ ]:
sample_data['success_rate'] = sample_data['successes'] / N_TRIALS

In [ ]:
sample_data.reset_index().plot.box(by='tree', column='success_rate', figsize=(15,6), color={'boxes':'green'})
plt.yscale('symlog', linthresh=0.01)
plt.xticks(orientation=90)
yticks = [0,
          #0.001,0.002,0.005,
          0.005,
          0.01,0.02,0.05,
          0.1,0.2,0.5,
          1.0]
plt.yticks(yticks, ['{0:.1f}%'.format(y*100) for y in yticks])
plt.ylim(0,1)
plt.grid()